## **Self RAG**

Self-RAGは、言語モデル（LM）によって生成されたテキストの精度と品質を向上させる手法です。関連情報を検索によって見つけ、モデルがその出力について振り返ることを可能にすることで、これを実現します。

モデルは検索された文章の助けを借りてテキストを生成し、その後、振り返りトークンを作成することで自身の回答をチェックします。これらのトークンは、モデルに対して、より多くの情報が必要かどうか、または回答が完全で検索されたデータによって裏付けられているかどうかを伝えます。

Self-RAGは、３つの採点者(Grader)を使います。
- document grader: 検索してきた文書が証拠として使えるか使えないか
- halucination grader: 回答に証拠があるかないか
- answer grader: 回答に妥当性があるかないか

Research Paper: [Self RAG](https://arxiv.org/pdf/2310.11511)

In [1]:
# !pip install --upgrade pip


In [55]:
import os
from dotenv import load_dotenv
load_dotenv()

# # for web search　今回は使わない
# os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

# VERTEXAI用の設定
import vertexai
import google.generativeai as genai

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("~/.config/gcloud/application_default_credentials.json")
vertexai.init(project=os.getenv("gcp_project_id"), location="us-central1")

# load llm
from langchain_google_vertexai import ChatVertexAI
llm = ChatVertexAI(
    model_name="gemini-2.0-flash-exp",
    project=os.getenv("gcp_project_id"),
    location="us-central1",
    temperature=0
)

# # load data
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf")
documents = loader.load()

# split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(documents)

# huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks... To disable this warning, you can either: - Avoid using tokenizers before the fork if possible - Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
os.environ["TOKENIZERS_PARALLELISM"] = "true" # 警告対策　tokenizersライブラリの並列処理を明示的にON 

# load embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    encode_kwargs={"normalize_embeddings": True}
)

from langchain.vectorstores import Chroma ## 何故か今日は以下のエラーが出てつかえなかった、機能は使えたのに。謎。仕方がないのでFAISSに変更した。
# RuntimeError: Chroma is running in http-only client mode, and can only be run with 'chromadb.api.fastapi.FastAPI' or 'chromadb.api.async_fastapi.AsyncFastAPI' as the chroma_api_impl.             see https://docs.trychroma.com/guides#using-the-python-http-only-client for more information.
# Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents, embeddings)
# vectorstore = Chroma.from_documents(documents, embeddings, persist_directory="../data/chroma_db_57")
# vectorstore = Chroma(persist_directory="../data/chroma_db_57", embedding_function=embeddings, client_settings=settings)
# vectorstore = Chroma(persist_directory="../data/chroma_db_57", embedding_function=embeddings)

# create retirever
retriever = vectorstore.as_retriever()

## **Document Grader**
ドキュメント評価者は、ドキュメントが与えられたクエリに関連しているかどうかを評価します。

In [56]:
# create grader for doc retriever
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI

# define a data class
class GradeDocuments(BaseModel): 
    """Schema for grading retrieved documents for relevance.
    The field 'binary_score' is expected to be either "yes" or "no" indicating
    whether the document is relevant to the user's question.
    """
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt for the grader
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [57]:
# testing grader
question = "沖縄科学技術大学院大学(OIST)のスタートアップ支援に関する予算額は？"
docs = retriever.invoke(question)
print(retrieval_grader.invoke({"question": question, "document": docs}))

binary_score='yes'


# RAG Chain

In [58]:
# create document chain
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """"
You are a helpful assistant that answers questions based on the following context
Context: {context}
Question: {question}
Answer:

"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = prompt | llm | StrOutputParser()

In [7]:
# response
generation = rag_chain.invoke({"context": docs, "question": question})
generation

'沖縄科学技術大学院大学(OIST)のスタートアップ支援に関する予算額は、R4補正で23億円の内数、R5補正で26億円の内数、R6当初で196億円の内数(内閣府)です。\n'

# Hallucination Grader
その答えが与えられた事実に基づいていたり、裏付けられているかどうかを確認する。

(先程のDocument Graderは retrieve したドキュメントが与えられたクエリに関連しているかどうかを評価していた。)

In [59]:
# create grader for hallucination
# define a data class
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# prompt for the grader
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='no')

## **Answer Grader**
回答が与えられた質問に効果的に対処しているかどうかを評価します。

In [60]:
# create grader for answer
# define a data class
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# prompt for the grader
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='no')

# Create Graph

## define Graph State

In [61]:
# define a data class for state
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    question: str
    generation: str
    documents: List[str]

In [ ]:
# nodeは update された state を返し、エッジは文字列を返します。

from pprint import pprint
import time

MAX_RETRIES = 3  

# ノード
def retrieve(state):
    print("---関連文書の取得---")
    question = state["question"]

    # 取得
    try:
        time.sleep(1)
        documents = retriever.invoke(question)
        state["documents"] = documents  # stateオブジェクトを直接変更
    except Exception as e:
        print(f"取得中のエラー: {e}")
        state["documents"] = []  # stateオブジェクトを直接変更
    return state


def generate(state):
    print("---回答生成---")
    question = state["question"]
    documents = state["documents"]

    # RAG生成
    try:
        time.sleep(1)
        generation = rag_chain.invoke({"context": documents, "question": question})
        state["generation"] = generation  # stateオブジェクトを直接変更
    except Exception as e:
        print(f"回答生成中のエラー: {e}")
        state["generation"] = ""  # stateオブジェクトを直接変更
    return state


def grade_documents(state):
    print("---質問に対する文書の関連性を確認---")
    question = state["question"]
    documents = state["documents"]

    # 各文書をスコアリング
    filtered_docs = []
    for d in documents:
        try:
            time.sleep(1)
            score = retrieval_grader.invoke(
                {"question": question, "document": d.page_content}
            )
            grade = score.binary_score
            if grade == "yes":
                print("---評価: 文書は関連しています---")
                filtered_docs.append(d)
            else:
                print("---評価: 文書は関連していません---")
        except Exception as e:
            print(f"文書評価中のエラー: {e}")
            continue
    state["documents"] = filtered_docs  # stateオブジェクトを直接変更
    return state


# エッジ
def decide_to_generate(state):
    print("---OKな文書があれば回答を生成---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # すべての文書がフィルタリングされました
        print("---決定: すべての文書は質問に関連していません---")
        return "no_relevant_documents"
    else:
        # 関連する文書があるので、回答を生成
        print("---決定: 生成---")
        return "generate"


def check_hallucination(state):  # 幻覚チェックの関数を分離
    print("---幻覚を確認---")
    documents = state["documents"]
    generation = state["generation"]
    try:
        time.sleep(1)
        score = hallucination_grader.invoke(
            {"documents": documents, "generation": generation}
        )
        grade = score.binary_score
        return grade == "yes"
    except Exception as e:
        print(f"幻覚評価中のエラー: {e}")
        return False


def evaluate_answer(state):  # 回答評価の関数を分離
    print("---生成と質問を評価---")
    question = state["question"]
    generation = state["generation"]
    try:
        time.sleep(1)
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        return grade == "yes"
    except Exception as e:
        print(f"回答評価中のエラー: {e}")
        return False


def grade_generation_v_documents_and_question(state):
    retry_count = state.get("retry_count", 0)
    print(f"{retry_count=}")
    if retry_count >= MAX_RETRIES:
        print(f"---最大再試行回数({MAX_RETRIES})に達しました---")
        return "max_retries_reached"

    if not check_hallucination(state):
        print("---決定: 生成は文書に基づいていません、再試行---")
        state["retry_count"] = retry_count + 1
        return "not supported"

    if not evaluate_answer(state):
        print("---決定: 生成は質問に答えていません---")
        return "not useful"

    print("---決定: 生成は文書に基づいており、質問に答えています---")
    return "useful"

このコードは、langgraph ライブラリを使ってワークフローグラフを構成するシンプルな例です。最初に、グラフの基本状態（GraphState）を元に StateGraph を作成し、開始（START）および終了（END）の定数とともにワークフローを定義しています。

主要な処理は「retrieve」「grade_documents」「generate」という 3 つのノードに分かれており、それぞれ対応する関数（retrieve、grade_documents、generate）を実行します。最初は START から「retrieve」へ、次に「retrieve」から「grade_documents」へと順に進みます。

そこから、grade_documents の結果に応じて条件付きで分岐し、decide_to_generate 関数が返す結果が「generate」の場合は「generate」ノードに、関連文書がない場合はワークフローを終了（END）します。さらに、generate ノードの出力を grade_generation_v_documents_and_question 関数で評価し、結果に応じて再試行（"not supported"）や終了（"not useful"、"max_retries_reached"、"useful"）の判断を下します。

最後に workflow.compile() で定義したグラフを実行可能なアプリケーション app にコンパイルしています。これにより、各ノードが順番に実行され、条件に基づいた分岐処理が行われるワークフローが完成します。

In [119]:
# Build graph
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "generate": "generate",
        "no_relevant_documents": END,
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "not useful": END,
        "max_retries_reached": END,
        "useful": END,
    },
)

# Compile
app = workflow.compile()

## Build Graph

In [120]:
# example 2 (no relevant documents)
from pprint import pprint

inputs = [
    {"question": "ディープテック・スタートアップの起業・経営人材確保等支援事業の予算額は？"},
]

for inp in inputs:
    for i, output in enumerate(app.stream(inp)):
        for key, value in output.items():  # i=0 key='retrieve' value.keys()=dict_keys(['documents', 'question'])
            pprint(f"Node '{key}':")

    if "generation" in value:
        pprint(value['generation'])
    else:
        pprint("関連文書が見つからない、または生成されない。")


---関連文書の取得---
"Node 'retrieve':"
---質問に対する文書の関連性を確認---
---評価: 文書は関連していません---
---評価: 文書は関連しています---
---評価: 文書は関連していません---
---評価: 文書は関連していません---
---OKな文書があれば回答を生成---
---決定: 生成---
"Node 'grade_documents':"
---回答生成---
retry_count=0
---幻覚を確認---
---生成と質問を評価---
---決定: 生成は質問に答えていません---
"Node 'generate':"
'ディープテック分野への横展開（NEDO）の予算として、15億円の内数（令和６年度当初予算）と21億円の内数（令和７年度概算要求額）が記載されています。'


## **Preparing Data for Evaluation**

In [148]:
# Create a dataframe to store the question, context, and response
import time

inputs = [
{"question": "グローバル・スタートアップ・アクセラレーションプログラムの予算額は？"},
{"question": "ディープテック・スタートアップの起業・経営人材確保等支援事業の予算額は？"},
{"question": "図書館間貸借制度はどのように機能していますか？"},
]

outputs = []

for inp in inputs:
    try:
        for output in app.stream(inp):
            # evaluation_results = {
            #     "document_relevance": [],
            #     "hallucination_check": False,
            #     "answers_question": False
            # }
            
            for key, value in output.items():
                if key == "generate":
                    question = value["question"]
                    documents = value["documents"]
                    generation = value["generation"]
                    
                    # コンテキストを文字列のリストとして保持
                    # contexts = [doc.page_content for doc in documents]

                    # check_relevance = output.get("document_relevance", [])
                    # # 評価結果を記録

                    # # ドキュメントの関連性（RELEVANT/NOT RELEVANTのリスト）
                    # evaluation_results["document_relevance"]
                
                    
                    # # 幻覚チェック結果
                    # evaluation_results["hallucination_check"] = "---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---" in str(output)
                    
                    # # 質問への適合性
                    # evaluation_results["answers_question"] = "---DECISION: GENERATION DOES NOT ADDRESS QUESTION---" not in str(output)
                    
                    # Append the result with ragas format and evaluation results
                    outputs.append({
                        "user_input": question,
                        "retrieved_contexts": [doc.page_content for doc in documents],
                        "response": generation,
                        # "evaluation": evaluation_results
                    })
    except Exception as e:
        if "Quota exceeded" in str(e):
            print("生成要求のクォータ上限に達しました。後ほど再実行するか、Vertex AIのクォータ増加を申請してください。")
        else:
            print(f"エラーが発生しました: {e}")
        # クォータ超過の場合は、少し待ってから次の質問へ進むか、もしくは処理をスキップ
        time.sleep(10)

---関連文書の取得---
---質問に対する文書の関連性を確認---
---評価: 文書は関連しています---
---評価: 文書は関連しています---
---評価: 文書は関連していません---
---評価: 文書は関連していません---
---OKな文書があれば回答を生成---
---決定: 生成---
---回答生成---
retry_count=0
---幻覚を確認---
---生成と質問を評価---
---決定: 生成は文書に基づいており、質問に答えています---
---関連文書の取得---
---質問に対する文書の関連性を確認---
---評価: 文書は関連していません---
---評価: 文書は関連しています---
---評価: 文書は関連していません---
---評価: 文書は関連していません---
---OKな文書があれば回答を生成---
---決定: 生成---
---回答生成---


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-experimental. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-experimental. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota excee

retry_count=0
---幻覚を確認---
---生成と質問を評価---


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-experimental. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-experimental. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota excee

---決定: 生成は質問に答えていません---
---関連文書の取得---
---質問に対する文書の関連性を確認---
---評価: 文書は関連していません---
---評価: 文書は関連していません---
---評価: 文書は関連していません---
---評価: 文書は関連していません---
---OKな文書があれば回答を生成---
---決定: すべての文書は質問に関連していません---


# Evaluate in RAGAS
今回は正解データがないのでFaithfulnessのみ
詳細 https://docs.ragas.io/en/stable/references/metrics/#ragas.metrics.Metric

In [155]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
# from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.metrics import Faithfulness
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_list(outputs)
evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    # metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    metrics=[Faithfulness()],
    llm=evaluator_llm,
)

result



Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'faithfulness': 1.0000}